Presented By: Team 2 - Aastha Surana, Jooyeon Lee, Zicheng Wang

# Starbucks Coffee Optimization – Full Model & Sensitivity Analysis
### BA885 – Phase II (Summer 2025)

---



# Problem Statement

Starbucks is under growing cost pressure as Arabica coffee prices hit a 47-year high of $4.40/lb in 2024, driven by global supply shortages and climate disruptions in key producing countries such as Brazil and Colombia.
(Source: [Bloomberg – Coffee Prices Soar as Climate Hits Brazil’s Arabica Supply](https://www.bloomberg.com/news/articles/2024-05-10/coffee-prices-soar-as-climate-hits-brazil-s-arabica-supply))

At the same time, over 75% of Starbucks’ beverage sales are now cold drinks, a major shift from its traditional hot coffee base. These cold beverages—primarily iced lattes, cold brews, and refreshers—tend to be less sensitive to bean origin and quality, as they are often diluted with milk, sweeteners, or ice, making taste differences less perceptible.

Starbucks has historically maintained a 100% Arabica policy, positioning itself as a premium coffee provider. However, the shift in consumer behavior toward cold drinks, especially among younger demographics, presents a strategic opportunity:

**What if Starbucks introduced Robusta beans for drinks, while preserving quality of beverages?**

Robusta beans offer several strategic advantages. They are 40–60% cheaper than Arabica, with current prices averaging $2.00–$2.50 per pound.
They also contain more caffeine and deliver a bolder flavor, making them well-suited for cold beverages that are mixed with milk or sweeteners.
In addition, Robusta is more climate-resilient, requiring less altitude and water, and is generally more productive per hectare than Arabica.

This shift could allow Starbucks to reduce costs, diversify sourcing, improve environmental sustainability, and better align its supply chain with the demands of its fastest-growing product segment.



# Objective

Minimize the total system cost of producing 100M+ beverages while satisfying Starbucks’ brand quality and operational constraints.  
We model the optimal blending of Arabica and Robusta across hot and cold drinks, considering:

- Bean sourcing costs
- Roasting preferences
- Caffeine levels
- Storage and transportation


# LP model Analysis

### **Step 0: Install Gurobi Solver**

In [ ]:
pip install gurobipy

In [ ]:
!pip install -i https://pypi.gurobi.com gurobipy

Looking in indexes: https://pypi.gurobi.com


### **Step 1: Import Necessary Libraries**

In [ ]:
from gurobipy import Model, GRB

### **Step 2: Define Data and Parameters for Starbucks Optimization**

**2.1 Coffee-Bean Purchase Costs ($/lb)**

Arabica beans are significantly more expensive, motivating the evaluation of introducing Robusta beans in certain beverages.

**Current Arabica bean price:** $4.40/lb (Source: [Perfect Daily Grind, Coffee News Recap – Feb 2025](https://perfectdailygrind.com/2025/02/coffee-news-recap-21-february-2025/))

**Current Robusta bean price:** $2.40/lb (Source: [YCharts, Coffee Robusta Price](https://perfectdailygrind.com/2025/02/coffee-news-recap-21-february-2025/))

In [ ]:
cost_green = {
    "Arabica": 4.40,  # Current Arabica bean price
    "Robusta": 2.40   # Current Robusta bean price
}

**2.2 Roasting Costs ($/lb roasted)**

Roasting costs differ by roast level due to variations in roasting time and energy requirements. Specific costs per roast type are industry standard assumptions.

**Blonde Roast:** $4.00/lb (industry estimate assumption, lighter roast costs slightly lower)

**Medium Roast:** $4.50/lb (industry estimate assumption)

**Dark Roast:** $5.00/lb (industry estimate assumption, higher cost due to longer roasting times)

In [ ]:
cost_roast = {
    "Blonde": 4.00,   # Blonde roast
    "Medium": 4.50,   # Medium roast
    "Dark":   5.00    # Dark roast
}

**2.3 Storage & Transport Costs**

Efficient logistics significantly impact total system costs. These costs are derived from U.S. transportation data and storage industry benchmarks.

**Storage cost:** $0.50 per pound (industry estimate assumption)


**Transport cost per ton-mile:**

- Land: $0.03 (Source: [Bureau of Transportation Statistics](https://www.bts.gov/content/average-freight-revenue-ton-mile))

- Ocean: $0.01 (Source: [Bureau of Transportation Statistics](https://www.bts.gov/content/average-freight-revenue-ton-mile))

- Air: $4.60, significantly higher cost due to air freight premium(Source: [Bureau of Transportation Statistics](https://www.bts.gov/content/average-freight-revenue-ton-mile))

**Conversion factor:** 1 ton = 2000 lbs (standard conversion)

In [ ]:
cost_storage_per_lb = 0.50  # Estimated storage cost per pound (industry estimate)

cost_transport_per_ton_mile = {
    "Land":  0.03,
    "Ocean": 0.01,
    "Air":   4.60
}
TON_TO_LB = 2000  # Conversion factor (2000 lbs = 1 ton)

**2.4 Annual Beverage Demand (cups → lbs)**

Total beverage demand is calculated based on Starbucks annual beverage sales and typical industry consumption patterns.

**Total annual cups:** 100,000,000 (Source: [Starbucks FY24 Annual Report](https://s203.q4cdn.com/326826266/files/doc_financials/2024/ar/Starbucks-Fiscal-2024-Annual-Report.pdf))

**Cold beverages proportion:** 75% of total beverages (Source: [QSR Magazine](https://www.qsrmagazine.com/food/beverage/cold-drinks-now-represent-75-percent-of-starbucks-sales/ ))

**Average coffee weight per cup:** 0.022 lbs per cup (Source: [Specialty Coffee Association](https://www.coffeefriend.co.uk/blog/how-much-ground-coffee-to-use-per-cup/?utm_source=chatgpt.com))

In [ ]:
cups_per_year   = 100_000_000        # Starbucks annual beverage sales (source: Starbucks Annual Report 2024)
pct_cold        = 0.75               # 75% of Starbucks U.S. sales are cold beverages (source: QSR Magazine)
avg_lbs_per_cup = 0.022              # Average weight of coffee beans per cup (industry standard assumption)

total_lbs = cups_per_year * avg_lbs_per_cup
hot_lbs   = total_lbs * (1 - pct_cold)
cold_lbs  = total_lbs * pct_cold

**2.5 Roast-Type Constraints**

These constraints ensure product variety and meet consumer preferences.

**Blonde roast maximum proportion:** 30% (Source: [Business Insider](https://www.businessinsider.com/starbucks-blonde-is-popular-2012-3))

**Minimum proportion per roast type (Blonde, Medium, Dark):** 10% each (operational assumption)

In [ ]:
max_blonde_pct   = 0.30
min_roast_pct    = 0.10

**2.6 Quality Thresholds for Beverages**

To preserve Starbucks’ brand quality, specific thresholds for Arabica content in beverages are set.

**Hot beverages:** Minimum 85% Arabica (operational and branding assumption)

**Cold beverages:** Maximum 50% Arabica (strategic assumption for cost reduction)

In [ ]:
min_arabica_hot  = 0.85
max_arabica_cold = 0.50

**2.7 Origin Sourcing Limits (Arabica Beans)**

To mitigate supply chain risks, Starbucks limits the proportion of beans sourced from a single origin.

**Origins considered:** Brazil, Colombia, Ethiopia, Vietnam (Source: [Supply Chain Xchange](https://www.thescxchange.com/article/starbucks-supply-chain))

**Maximum sourcing from any single origin:** 40% (Source: [Starbucks FY24 Annual Report](https://s203.q4cdn.com/326826266/files/doc_financials/2024/ar/Starbucks-Fiscal-2024-Annual-Report.pdf))

In [ ]:
origins        = ["Brazil", "Colombia", "Ethiopia", "Vietnam"]
max_origin_pct = 0.40

**2.8 Storage & Transport Budget Constraints**

Budget constraints based on operational capacity and financial planning:

**Maximum storage capacity:** 20,000,000 lbs (Source: [Starbucks FY24 Annual Report](https://s203.q4cdn.com/326826266/files/doc_financials/2024/ar/Starbucks-Fiscal-2024-Annual-Report.pdf))

**Maximum annual transport budget:** $300,000,000 (operational assumption based on industry logistics benchmarks)

In [ ]:
max_storage_lbs    = 20_000_000
max_transport_cost = 300_000_000

### **Step 3: Model Setup**

We initialize a Gurobi model named "Starbucks_Blending". This model will help us determine the most cost-effective sourcing, roasting, and distribution plan that satisfies all business and operational constraints.

In [ ]:
# 3) MODEL SETUP
m = Model("Starbucks_Blending")
m.Params.OutputFlag = 0   # suppress solver output

### **Step 4: Decision Variables**

This section defines the decision variables of the optimization model. Each variable represents a key operational quantity in Starbucks' supply chain and production planning. All variables are in pounds (lbs) and constrained to be non-negative.

**4.1 Green-Bean Sourcing**

We decide how many pounds of Arabica and Robusta green coffee beans to purchase.
Arabica is preferred for quality but is more expensive, while Robusta is cheaper and more suitable for cold beverages or blends.

In [ ]:
aarabica = m.addVar(name="aarabica", lb=0)   # Total lbs of Arabica beans to purchase
arobusta = m.addVar(name="arobusta", lb=0)   # Total lbs of Robusta beans to purchase

In [ ]:
# Arabica and Robusta used in each beverage type
arabica_hot  = m.addVar(name="arabica_hot",  lb=0)
robusta_hot  = m.addVar(name="robusta_hot",  lb=0)
arabica_cold = m.addVar(name="arabica_cold", lb=0)
robusta_cold = m.addVar(name="robusta_cold", lb=0)


**4.2 Roasted Coffee by Roast Type**

We track how much of each roast type we produce after roasting the green beans. These will be blended into beverages.

In [ ]:
r_blonde = m.addVar(name="r_blonde", lb=0)   # Blonde roast output
r_medium = m.addVar(name="r_medium", lb=0)   # Medium roast output
r_dark   = m.addVar(name="r_dark",   lb=0)   # Dark roast output

**4.3 Beverage Usage (Hot vs Cold)**

The model distinguishes between hot and cold beverage volumes. This distinction is important because quality and blend standards differ across them.

In [ ]:
h = m.addVar(name="hot_lbs",  lb=0)   # Coffee used in hot beverages
c = m.addVar(name="cold_lbs", lb=0)   # Coffee used in cold beverages

**4.4 Caffeine Levels**

To serve a diverse customer base, we must split coffee volume into three caffeine-level categories.

In [ ]:
c1 = m.addVar(name="caf_lbs",    lb=0)  # Fully caffeinated
c2 = m.addVar(name="lowcaf_lbs", lb=0)  # Low caffeine
c3 = m.addVar(name="decaf_lbs",  lb=0)  # Decaffeinated

**4.5 Arabica Sourcing by Country**

Arabica must be sourced from specific countries. We track origin-wise sourcing to ensure diversity and manage risk.

In [ ]:
s = {o: m.addVar(name=f"s_{o}", lb=0) for o in origins}

Where origins = ["Brazil", "Colombia", "Ethiopia", "Vietnam"]

**4.6 Roasted Coffee in Storage**

A portion of roasted coffee may be stored to meet inventory policies or account for transport delays.

In [ ]:
w = m.addVar(name="storage_lbs", lb=0)

**4.7 Transportation by Mode**

We must decide how to allocate transportation volume across land, ocean, and air routes. Each has different costs and implications.

In [ ]:
t_land  = m.addVar(name="t_land_lbs",  lb=0)
t_ocean = m.addVar(name="t_ocean_lbs", lb=0)
t_air   = m.addVar(name="t_air_lbs",   lb=0)

### **Step 5: Objective Function**

The objective of our optimization model is to minimize the total cost of producing and delivering 100 million cups of coffee. The cost components include:

1. **Green Bean Sourcing Cost:** We pay per pound for Arabica and Robusta beans.

- Arabica: $4.40/lb

- Robusta: $2.40/lb

- Roasting Cost

2. **Roasting beans** incurs cost depending on the roast level.

- Blonde: $4.00/lb

- Medium: $4.50/lb

- Dark: $5.00/lb

  These are illustrative assumptions based on industry norms.

3. **Storage Cost**

  Stored roasted coffee costs $0.50/lb. This could include warehousing, inventory holding, and spoilage risk.

4. **Transportation Cost**

  Transportation is modeled as cost per ton-mile. We assume an average distance of 1,000 miles per shipment:

- Land: $0.03/ton-mile

- Ocean: $0.01/ton-mile

- Air: $4.60/ton-mile

  Transportation cost is scaled from pounds to tons using a conversion factor (1 ton = 2,000 lbs).

**Objective:** Minimize total cost:
```
Z = 4.4 * aarabica + 2.4 * arobusta
  + 4.0 * rblonde + 4.5 * rmedium + 5.0 * rdark
  + 0.5 * w
  + 0.03 * tland + 0.01 * tocean + 4.6 * tair
```

By minimizing this cost function, Starbucks can meet product demand while choosing the most cost-effective blend of beans, roast levels, storage strategies, and transport modes — all under realistic constraints.

In [ ]:
m.setObjective(
      cost_green["Arabica"] * aarabica
    + cost_green["Robusta"] * arobusta
    + cost_roast["Blonde"] * r_blonde
    + cost_roast["Medium"] * r_medium
    + cost_roast["Dark"]   * r_dark
    + cost_storage_per_lb * w
    + cost_transport_per_ton_mile["Land"]  * (t_land  / TON_TO_LB) * 1000
    + cost_transport_per_ton_mile["Ocean"] * (t_ocean / TON_TO_LB) * 1000
    + cost_transport_per_ton_mile["Air"]   * (t_air   / TON_TO_LB) * 1000,
    GRB.MINIMIZE
)

### **Step 6: Constraints**

To ensure feasibility and practicality, our optimization model includes several business-driven constraints:

**6.1 Demand Fulfillment**

We must satisfy total annual demand:

- hot_lbs: must match the demand for hot coffee

- cold_lbs: must match the demand for cold coffee

These values are calculated from 100M cups × 2.2% lbs per cup and a 75% cold mix assumption.

In [ ]:
m.addConstr(h == hot_lbs,  name="Hot_Demand")
m.addConstr(c == cold_lbs, name="Cold_Demand")

<gurobi.Constr *Awaiting Model Update*>

In [ ]:
m.addConstr(arabica_hot + arabica_cold == aarabica, name="ArabicaUsage")
m.addConstr(robusta_hot + robusta_cold == arobusta, name="RobustaUsage")


<gurobi.Constr *Awaiting Model Update*>

In [ ]:
# Arabica + Robusta must satisfy beverage demand
m.addConstr(arabica_hot + robusta_hot == h, name="Hot_Blend_Balance")
m.addConstr(arabica_cold + robusta_cold == c, name="Cold_Blend_Balance")


<gurobi.Constr *Awaiting Model Update*>

**6.2 Cold-to-Hot Ratio**

As per consumption trends, cold beverages account for 75% of demand, establishing a 3:1 cold-to-hot ratio:

c == 3 * h

In [ ]:
m.addConstr(c == 3 * h, name="Cold_Hot_Ratio")

<gurobi.Constr *Awaiting Model Update*>

**6.3 Roast Balance**

All roasted coffee (blonde, medium, dark) must add up to total consumption (hot + cold):

r_blonde + r_medium + r_dark == h + c

In [ ]:
m.addConstr(r_blonde + r_medium + r_dark == h + c, name="Roast_Balance")

<gurobi.Constr *Awaiting Model Update*>

**6.4 Bean Balance**

The amount of green beans sourced (Arabica + Robusta) must equal the total roasted output:

aarabica + arobusta == r_blonde + r_medium + r_dark

In [ ]:
m.addConstr(aarabica + arobusta == r_blonde + r_medium + r_dark, name="Bean_Balance")

<gurobi.Constr *Awaiting Model Update*>

**6.5 Roast Composition Caps**

To maintain product diversity:

- Each roast must be ≥10% of the total.

- Blonde roast is capped at 30%, reflecting customer preference or flavor profile concerns.

In [ ]:
total_roast = r_blonde + r_medium + r_dark
m.addConstr(r_blonde >= min_roast_pct * total_roast, name="Blonde_Min")
m.addConstr(r_medium >= min_roast_pct * total_roast, name="Medium_Min")
m.addConstr(r_dark   >= min_roast_pct * total_roast, name="Dark_Min")
m.addConstr(r_blonde <= max_blonde_pct * total_roast, name="Blonde_Max")

<gurobi.Constr *Awaiting Model Update*>

**6.6 Arabica Quality Requirements**

Quality targets by beverage type:

- Hot drinks: must contain ≥85% Arabica

- Cold drinks: can contain ≤50% Arabica

This ensures taste and brand consistency.

In [ ]:
m.addConstr(arabica_hot >= min_arabica_hot * h, name="Hot_Arabica_Quality")
m.addConstr(arabica_cold <= max_arabica_cold * c, name="Cold_Arabica_Quality")


<gurobi.Constr *Awaiting Model Update*>

**6.7 Caffeine Level Balance**

Cups must be assigned to:

- Caffeinated (c1), Low-caffeine (c2), Decaf (c3)

Also, at least 10% must be low-caf or decaf, to meet market trends and health-conscious consumer preferences.

In [ ]:
m.addConstr(c1 + c2 + c3 == h + c,                        name="CafBalance")
m.addConstr(c2 + c3 >= min_roast_pct * (c1 + c2 + c3),    name="LowDecaf_Min")

<gurobi.Constr *Awaiting Model Update*>

**6.8 Origin-Based Arabica Sourcing**

To ensure supplier diversity and resilience:

- Each origin (Brazil, Colombia, Ethiopia, Vietnam) must contribute ≤40% of total Arabica volume.



In [ ]:
m.addConstr(sum(s[o] for o in origins) == aarabica,      name="Arabica_Origin_Sum")
for o in origins:
    m.addConstr(s[o] <= max_origin_pct * aarabica,       name=f"OriginCap_{o}")

**6.9 Inventory Storage**

To maintain buffer stock:

- At least 20% of roasted coffee must go into storage.

- Also limited to max capacity (20M lbs).

In [ ]:
m.addConstr(w >= 0.20 * (h + c), name="Min_Inventory")
m.addConstr(w <= max_storage_lbs,   name="Storage_Limit")

<gurobi.Constr *Awaiting Model Update*>

**6.10 Transport Logic**

- The sum of transport modes (land, ocean, air) must match total green beans sourced.

- Total cost must stay under $300M, enforcing budget discipline.

In [ ]:
m.addConstr(t_land + t_ocean + t_air == aarabica + arobusta, name="Transport_Balance")
transport_cost = (
      cost_transport_per_ton_mile["Land"]  * (t_land  / TON_TO_LB) * 1000
    + cost_transport_per_ton_mile["Ocean"] * (t_ocean / TON_TO_LB) * 1000
    + cost_transport_per_ton_mile["Air"]   * (t_air   / TON_TO_LB) * 1000
)
m.addConstr(transport_cost <= max_transport_cost, name="Transport_Budget")

<gurobi.Constr *Awaiting Model Update*>

**6.11 Minimum Mode Utilization**

Each transport method must carry at least 10% of the volume — to ensure supplier flexibility and risk mitigation.

In [ ]:
total_transport = aarabica + arobusta
m.addConstr(t_land  >= 0.10 * total_transport, name="Min_Transport_Land")
m.addConstr(t_ocean >= 0.10 * total_transport, name="Min_Transport_Ocean")
m.addConstr(t_air   >= 0.10 * total_transport, name="Min_Transport_Air")

<gurobi.Constr *Awaiting Model Update*>

### **Step 7: Solve the Model**

With the objective function and constraints fully defined, we now invoke the Gurobi solver to find the optimal solution.

In [ ]:
m.optimize()

### **Step 8: Interpret the Output**

**Optimal Cost:** Displays the minimized total cost of coffee production, roasting, storage, and transportation.

1. **Demand Split: Hot vs. Cold**

  Reports the breakdown of total demand between hot and cold beverage based on fixed assumptions: 75% cold, 25% hot.

2. **Bean Blend by Beverage Type**

  Details the composition of Arabica vs. Robusta in hot and cold beverages:

  Hot drinks are constrained to have ≥85% Arabica.

  Cold drinks can have ≤50% Arabica.

  We use these thresholds to estimate and report the approximate split of bean types within each beverage category.

3. **Overall Bean Sourcing:** Shows the total quantity of Arabica and Robusta sourced and their proportion of total output.

4. **Roast Breakdown:** Provides the total amount roasted per roast level (Blonde, Medium, Dark), along with each type's share of the overall volume.

5. **Inventory:** Displays the volume stored in inventory and its proportion relative to total roasted volume.

  Note: Business rule requires ≥20% inventory at any time.

6. **Transportation Mode:** Breakdown of logistics by transportation method (Land, Ocean, Air), including:

- Absolute weight transported.

- Each mode’s % of total product movement.

7. **Origin Mix**

  Reports sourcing of Arabica from each origin country (Brazil, Colombia, Ethiopia, Vietnam), both in pounds and as a share of total production.

  This is important for compliance with supply diversity limits (no origin >40%).

In [ ]:
if m.status == GRB.OPTIMAL:
    total_output = h.x + c.x

    print(f"\n Optimal cost: ${m.objVal:,.0f}\n")

    # Hot & Cold breakdown
    print("Demand split:")
    for var in [h, c]:
        pct = var.x / total_output * 100
        print(f"  {var.varName:<12s} = {var.x:,.0f} lbs ({pct:.1f}%)")

    print("\n Hot beverages blend:")
    print(f"  Arabica_hot  = {arabica_hot.x:,.0f} lbs ({arabica_hot.x / h.x * 100:.1f}%)")
    print(f"  Robusta_hot  = {robusta_hot.x:,.0f} lbs ({robusta_hot.x / h.x * 100:.1f}%)")

    print("\n Cold beverages blend:")
    print(f"  Arabica_cold = {arabica_cold.x:,.0f} lbs ({arabica_cold.x / c.x * 100:.1f}%)")
    print(f"  Robusta_cold = {robusta_cold.x:,.0f} lbs ({robusta_cold.x / c.x * 100:.1f}%)")


    # Beans
    print("\nBeans overall:")
    for var in [aarabica, arobusta]:
        pct = var.x / total_output * 100
        print(f"  {var.varName:<12s} = {var.x:,.0f} lbs ({pct:.1f}%)")

    # Roast
    print("\nRoast:")
    for var in [r_blonde, r_medium, r_dark]:
        pct = var.x / total_output * 100
        print(f"  {var.varName:<12s} = {var.x:,.0f} lbs ({pct:.1f}%)")

    # Storage
    pct_w = w.x / total_output * 100
    print(f"\nInventory:       {w.x:,.0f} lbs ({pct_w:.1f}%)")

    # Transport
    print("\nTransport:")
    for var in [t_land, t_ocean, t_air]:
        pct = var.x / total_output * 100
        print(f"  {var.varName:<12s} = {var.x:,.0f} lbs ({pct:.1f}%)")

    # Origin mix
    print("\nOrigin‐mix:")
    for o in origins:
        pct = s[o].x / total_output * 100
        print(f"  {o:<10s} = {s[o].x:,.0f} lbs ({pct:.1f}%)")
else:
    print("⚠️ No optimal solution found.")



 Optimal cost: $16,633,100

Demand split:
  hot_lbs      = 550,000 lbs (25.0%)
  cold_lbs     = 1,650,000 lbs (75.0%)

 Hot beverages blend:
  Arabica_hot  = 467,500 lbs (85.0%)
  Robusta_hot  = 82,500 lbs (15.0%)

 Cold beverages blend:
  Arabica_cold = 0 lbs (0.0%)
  Robusta_cold = 1,650,000 lbs (100.0%)

Beans overall:
  aarabica     = 467,500 lbs (21.2%)
  arobusta     = 1,732,500 lbs (78.8%)

Roast:
  r_blonde     = 660,000 lbs (30.0%)
  r_medium     = 1,320,000 lbs (60.0%)
  r_dark       = 220,000 lbs (10.0%)

Inventory:       440,000 lbs (20.0%)

Transport:
  t_land_lbs   = 220,000 lbs (10.0%)
  t_ocean_lbs  = 1,760,000 lbs (80.0%)
  t_air_lbs    = 220,000 lbs (10.0%)

Origin‐mix:
  Brazil     = 187,000 lbs (8.5%)
  Colombia   = 187,000 lbs (8.5%)
  Ethiopia   = 93,500 lbs (4.2%)
  Vietnam    = 0 lbs (0.0%)


### **Interpretation of Optimization Results**

1. **Optimal Cost: $16,633,100**

 This represents the total minimum cost to meet Starbucks’ projected annual demand of 100 million cups, including green bean sourcing, roasting, storage, and transportation.

2. **Business Implication:**

This solution provides a lower-cost scenario by aggressively shifting Robusta use in cold beverages while maintaining Arabica quality in hot drinks. It serves as a cost-optimized strategy that prioritizes cost-efficiency in segments less sensitive to bean origin, offering a compelling trade-off for future cost-saving initiatives.

3. **Demand Split**

- Hot beverages: 550,000 lbs (25%)

- Cold beverages: 1,650,000 lbs (75%)

  Reflects Starbucks’ increasing emphasis on cold beverages, aligning with recent consumer trends. Cold products now dominate sales, so sourcing and quality strategies must prioritize this segment.

4. **Hot Beverage Blend**
- Arabica: 467,500 lbs (85%)

- Robusta: 82,500 lbs (15%)

  Hot drinks maintain a high Arabica share to preserve Starbucks' premium image and flavor profile, while allowing a small Robusta inclusion to marginally reduce costs without significant quality compromise.

5. **Cold Beverage Blend**
- Arabica: 0 lbs (0%)

- Robusta: 1,650,000 lbs (100%)

  Cold beverages use 100% Robusta, reflecting full cost minimization in this segment. Given the dilution and flavor masking in cold drinks, this aggressive shift provides major savings while maintaining customer satisfaction.

6. **Bean Sourcing – Overall**

- Arabica: 467,500 lbs (21.2%)

- Robusta: 1,732,500 lbs (78.8%)

 This is the most Robusta-heavy strategy among the evaluated scenarios. The model significantly reduces Arabica reliance while staying compliant with hot beverage quality constraints, achieving an optimal cost profile.

7. **Roast Mix**

- Blonde: 660,000 lbs (30%)

- Medium: 1,320,000 lbs (60%)

- Dark: 220,000 lbs (10%)

 Roast mix remains diversified with a focus on medium roast, which appeals to a broad customer base. Blonde is capped at 30%, per constraints, balancing roast variety and operational feasibility.

8. **Inventory: 440,000 lbs (20%)**

  Meeting the 20% minimum storage target ensures inventory resilience, supports demand variability, and minimizes stockouts during disruptions. It also allows Starbucks to plan roasting and distribution more efficiently.

9. **Transport Mix**

- Land: 220,000 lbs (10%)

- Ocean: 1,760,000 lbs (80%)

- Air: 220,000 lbs (10%)

 Ocean transport dominates due to its low cost. Land and air transport fulfill their 10% minimums, supporting regions with faster delivery needs or limited access, while keeping overall logistics costs in check.

10. **Arabica Origin Mix**

- Brazil: 187,000 lbs (8.5%)

- Colombia: 187,000 lbs (8.5%)

- Ethiopia: 93,500 lbs (4.2%)

- Vietnam: 0 lbs (0%)

  All origin shares stay within the 40% cap to promote supplier diversification. Ethiopia is used in a limited amount, potentially for premium blends. Vietnam is excluded here, likely because Robusta is used for cold drinks and Vietnam-origin Arabica was not prioritized under the current constraints.

# **Business Implications**
The optimized model delivers a total cost of $16,633,100 to meet Starbucks' projected annual demand of 100 million cups of coffee. This solution satisfies all sourcing, roast, inventory, and transportation constraints. However, the result reflects the model's current threshold configuration—particularly its loose cold beverage quality constraint—which directly shaped this cost-minimized strategy.

### **Strategic Shift to a Blended Sourcing Model**

The model employs a highly cost-efficient sourcing strategy:

Hot beverages (25% of volume):
85% Arabica / 15% Robusta, meeting the minimum Arabica quality requirement.

Cold beverages (75% of volume):
100% Robusta, due to the absence of a minimum Arabica constraint.

 Overall bean mix:

- Arabica: 467,500 lbs (21.2%)

- Robusta: 1,732,500 lbs (78.8%)

Interpretation:
This is not necessarily the most realistic or consumer-aligned blend—but rather the logical outcome of the model thresholds we imposed. The model simply exploited its freedom to minimize cost, given we allowed Arabica_cold ≥ 0%.

**Implication: Our current result is a cost floor—it shows the maximum possible savings if Starbucks fully replaces Arabica in cold beverages. But brand perception, customer tolerance, and taste profiles may require tighter thresholds.**



### **Geographic Diversification and Supply Chain Resilience**
Arabica sourcing is spread across:

- Brazil: 187,000 lbs (8.5%)

- Colombia: 187,000 lbs (8.5%)

- Ethiopia: 93,500 lbs (4.2%)

- Vietnam: 0 lbs

All origins comply with the 40% cap, supporting supplier diversification. Ethiopia is minimally used, possibly due to high cost per lb. Vietnam, usually a Robusta source, is not used for Arabica under current model decisions.


### **Roast and Inventory Strategy Optimization**
Roast allocation:

- Medium: 1,320,000 lbs (60%) — the most popular and cost-effective option.

- Blonde: 660,000 lbs (30%) — meets the model-imposed maximum.

- Dark: 220,000 lbs (10%) — ensures portfolio diversity.

Inventory:

440,000 lbs stored (20%) to meet minimum reserve requirements, absorb demand fluctuations, and support continuous roasting operations.

Implication: Roast and inventory planning align with consumer preferences and ensure supply chain resilience without excessive overhead.

### **Transport Efficiency**
Transport modes:

- Ocean: 1,760,000 lbs (80%) — low-cost, ideal for bulk shipments.

- Land: 220,000 lbs (10%)

- Air: 220,000 lbs (10%)

Ocean freight is prioritized for cost reasons, while land and air meet minimum thresholds for flexibility and coverage.



### **Strategic Takeaways**

This model shows the maximum savings possible when cold beverage quality constraints are loosened. The results should not be interpreted as the “ideal” business solution, but as a benchmark.

Sensitivity analysis on cold Arabica minimums is essential to find a realistic balance between cost and taste. Starbucks could use this result to evaluate the financial impact of raising quality standards back to 10–50% Arabica in cold drinks.


#Sensitivity Analysis

In [ ]:
if m.status == GRB.OPTIMAL:
    print("\n🔹 Variable Sensitivity Report")
    print(f"{'Variable':>25s} {'Value':>12s} {'Reduced Cost':>15s}")
    for v in m.getVars():
        print(f"{v.varName:>25s} {v.x:12,.1f} {v.RC:15,.4f}")

    print("\n🔹 Constraint Sensitivity Report")
    print(f"{'Constraint':>25s} {'Shadow Price':>15s} {'Slack':>12s}")
    for c in m.getConstrs():
        print(f"{c.ConstrName:>25s} {c.Pi:15,.4f} {c.Slack:12,.1f}")
else:
    print("⚠️ No optimal solution found.")



🔹 Variable Sensitivity Report
                 Variable        Value    Reduced Cost
                 aarabica    467,500.0          0.0000
                 arobusta  1,732,500.0          0.0000
              arabica_hot    467,500.0          0.0000
              robusta_hot     82,500.0          0.0000
             arabica_cold          0.0          2.0000
             robusta_cold  1,650,000.0          0.0000
                 r_blonde    660,000.0          0.0000
                 r_medium  1,320,000.0          0.0000
                   r_dark    220,000.0          0.0000
                  hot_lbs    550,000.0          0.0000
                 cold_lbs  1,650,000.0          0.0000
                  caf_lbs          0.0          0.0000
               lowcaf_lbs  2,200,000.0          0.0000
                decaf_lbs          0.0          0.0000
                 s_Brazil    187,000.0          0.0000
               s_Colombia    187,000.0          0.0000
               s_Ethiopia     93,5

The sensitivity analysis confirms that Starbucks’ current coffee sourcing and production plan is tightly optimized—many constraints are binding, meaning they are actively limiting the solution and directly impacting cost. The model is operating at the feasible edge of quality, inventory, and logistics trade-offs.

**Key Insights**


1.    Demand Constraints

- Hot_Demand (Shadow Price: 8.8355)
  Each additional pound of hot beverage would increase total cost by $8.84, reflecting the high cost of hot blends (mostly Arabica). This constraint is binding, meaning hot demand is being exactly met with no slack.

- Cold_Demand (Shadow Price: 7.1355)
  Each extra pound of cold beverage increases cost by $7.14—lower than hot drinks due to 100% Robusta usage. This constraint is also tight and fully utilized.

2.   Air Transport

- Min_Transport_Air (Shadow Price: 2.2950)
  Forcing the use of air transport adds $2.30/lb to cost. This mode is costly but necessary to meet the minimum 10% constraint. Any relaxation here would yield cost savings.

3.  Roast Composition

- Dark_Min (Shadow Price: 0.5000)
  The model just meets the minimum dark roast requirement. Adding more would raise cost by $0.50/lb, implying dark roast is relatively more expensive.

- Blonde_Max (Shadow Price: -0.5000)
  The cap on blonde roast (30%) is binding. If the cap were relaxed, cost could decrease by $0.50/lb, as blonde roast could offer better optimization in the blend.

- Medium_Min and Blonde_Min (Slack: large negative)
  These minimums are not binding—the model already exceeds these requirements, suggesting the current roast mix aligns well with cost and consumer preference.

4. Bean Composition
- Arabica_cold (Reduced Cost: 2.0000)
  Arabica is currently not used at all in cold drinks. Introducing 1 lb of Arabica_cold would increase total cost by $2.00, indicating a strong cost-saving incentive to keep cold drinks Robusta-only.

- Hot_Arabica_Quality (Shadow Price: 2.0000)
  The 85% Arabica minimum in hot drinks is binding. Increasing this threshold further would raise costs by $2.00/lb, confirming the model is pushing against quality constraints.

5. Inventory
- Min_Inventory (Shadow Price: 0.5000)

  The 20% inventory requirement increases cost by $0.50/lb, but provides essential supply resilience and demand buffering. A lower threshold could yield small savings but increase operational risk.

6. Non-Binding Constraints
- Storage_Limit, Transport_Budget, Min_Transport_Ocean
These constraints have large slack values and are not currently restrictive. This provides Starbucks with scaling flexibility in transport, warehouse capacity, and future supply changes.

- CafBalance, LowDecaf_Min, Arabica_Origin_Sum
These constraints are also non-binding, meaning the current solution already satisfies health and sourcing policies without pushing boundaries.

The model is finely tuned for cost efficiency within the current rules.
If Starbucks wishes to explore more realistic quality thresholds, particularly for cold beverages, it must accept moderate cost increases.
Meanwhile, relaxing caps on blonde roast, minimum air transport, or inventory could unlock additional savings.

#Scenario Analysis

In [ ]:
import pandas as pd

# Constants that do not change per scenario
cost_roast = {"Blonde": 4.00, "Medium": 4.50, "Dark": 5.00}
cost_storage_per_lb = 0.50
cost_transport_per_ton_mile = {"Land": 0.03, "Ocean": 0.01, "Air": 4.60}
TON_TO_LB = 2000

# Demand & roasting assumptions
cups_per_year = 100_000_000
pct_cold = 0.75
avg_lbs_per_cup = 0.022
total_lbs = cups_per_year * avg_lbs_per_cup
hot_lbs = total_lbs * (1 - pct_cold)
cold_lbs = total_lbs * pct_cold

# Roast proportions
min_roast_pct = 0.10
max_blonde_pct = 0.30

# Arabica Cold Quality (fixed at 50%)
max_arabica_cold = 0.50

# Origin and inventory limits
origins = ["Brazil", "Colombia", "Ethiopia", "Vietnam"]
max_origin_pct = 0.40
max_storage_lbs = 20_000_000
max_transport_cost = 300_000_000

# Scenario ranges
arabica_prices = [4.40, 5.00, 5.50, 6.00]
hot_qual_thresholds = [0.85, 0.80, 0.70, 0.60]

results = []

# Scenario loop
for price in arabica_prices:
    for hot_threshold in hot_qual_thresholds:
        print(f"\n Scenario: Arabica ${price:.2f}/lb | Hot Arabica ≥ {hot_threshold:.0%}")

        # Model setup
        m = Model("StarbucksScenario")
        m.Params.OutputFlag = 0

        # Green bean cost
        cost_green = {"Arabica": price, "Robusta": 2.40}

        # Decision variables
        aarabica = m.addVar(name="aarabica", lb=0)
        arobusta = m.addVar(name="arobusta", lb=0)

        arabica_hot  = m.addVar(name="arabica_hot",  lb=0)
        robusta_hot  = m.addVar(name="robusta_hot",  lb=0)
        arabica_cold = m.addVar(name="arabica_cold", lb=0)
        robusta_cold = m.addVar(name="robusta_cold", lb=0)

        r_blonde = m.addVar(name="r_blonde", lb=0)
        r_medium = m.addVar(name="r_medium", lb=0)
        r_dark   = m.addVar(name="r_dark",   lb=0)

        h = m.addVar(name="hot_lbs",  lb=0)
        c = m.addVar(name="cold_lbs", lb=0)

        c1 = m.addVar(name="caf_lbs",    lb=0)
        c2 = m.addVar(name="lowcaf_lbs", lb=0)
        c3 = m.addVar(name="decaf_lbs",  lb=0)

        s = {o: m.addVar(name=f"s_{o}", lb=0) for o in origins}

        w = m.addVar(name="storage_lbs", lb=0)

        t_land  = m.addVar(name="t_land_lbs",  lb=0)
        t_ocean = m.addVar(name="t_ocean_lbs", lb=0)
        t_air   = m.addVar(name="t_air_lbs",   lb=0)

        # Objective function
        m.setObjective(
              cost_green["Arabica"] * aarabica
            + cost_green["Robusta"] * arobusta
            + cost_roast["Blonde"] * r_blonde
            + cost_roast["Medium"] * r_medium
            + cost_roast["Dark"]   * r_dark
            + cost_storage_per_lb * w
            + cost_transport_per_ton_mile["Land"]  * (t_land  / TON_TO_LB) * 1000
            + cost_transport_per_ton_mile["Ocean"] * (t_ocean / TON_TO_LB) * 1000
            + cost_transport_per_ton_mile["Air"]   * (t_air   / TON_TO_LB) * 1000,
            GRB.MINIMIZE
        )

        # Constraints
        m.addConstr(h == hot_lbs,  name="Hot_Demand")
        m.addConstr(c == cold_lbs, name="Cold_Demand")

        m.addConstr(arabica_hot + arabica_cold == aarabica)
        m.addConstr(robusta_hot + robusta_cold == arobusta)
        m.addConstr(arabica_hot + robusta_hot == h)
        m.addConstr(arabica_cold + robusta_cold == c)
        m.addConstr(c == 3 * h)

        m.addConstr(r_blonde + r_medium + r_dark == h + c)
        m.addConstr(aarabica + arobusta == r_blonde + r_medium + r_dark)

        total_roast = r_blonde + r_medium + r_dark
        m.addConstr(r_blonde >= min_roast_pct * total_roast)
        m.addConstr(r_medium >= min_roast_pct * total_roast)
        m.addConstr(r_dark   >= min_roast_pct * total_roast)
        m.addConstr(r_blonde <= max_blonde_pct * total_roast)

        # Quality constraints (SCENARIO CONTROLLED)
        m.addConstr(arabica_hot  >= hot_threshold * h, name="Hot_Arabica_Quality")
        m.addConstr(arabica_cold <= max_arabica_cold * c)

        m.addConstr(c1 + c2 + c3 == h + c)
        m.addConstr(c2 + c3 >= min_roast_pct * (c1 + c2 + c3))

        m.addConstr(sum(s[o] for o in origins) == aarabica)
        for o in origins:
            m.addConstr(s[o] <= max_origin_pct * aarabica)

        m.addConstr(w >= 0.20 * (h + c))
        m.addConstr(w <= max_storage_lbs)

        m.addConstr(t_land + t_ocean + t_air == aarabica + arobusta)

        transport_cost = (
              cost_transport_per_ton_mile["Land"]  * (t_land  / TON_TO_LB) * 1000
            + cost_transport_per_ton_mile["Ocean"] * (t_ocean / TON_TO_LB) * 1000
            + cost_transport_per_ton_mile["Air"]   * (t_air   / TON_TO_LB) * 1000
        )
        m.addConstr(transport_cost <= max_transport_cost)

        total_transport = aarabica + arobusta
        m.addConstr(t_land  >= 0.10 * total_transport)
        m.addConstr(t_ocean >= 0.10 * total_transport)
        m.addConstr(t_air   >= 0.10 * total_transport)

        # Solve
        m.optimize()

        if m.status == GRB.OPTIMAL:
            results.append({
                "Arabica Price": price,
                "Hot Arabica Min %": hot_threshold,
                "Total Cost": m.objVal,
                "Arabica (lbs)": aarabica.x,
                "Robusta (lbs)": arobusta.x,
                "Arabica Cold %": arabica_cold.x / c.x if c.x > 0 else 0,
                "Arabica Hot %": arabica_hot.x / h.x if h.x > 0 else 0,
                "Storage (lbs)": w.x,
                "T_Land": t_land.x,
                "T_Ocean": t_ocean.x,
                "T_Air": t_air.x
            })
        else:
            print("⚠️ Infeasible scenario!")

# Show result table
df = pd.DataFrame(results)
df


 Scenario: Arabica $4.40/lb | Hot Arabica ≥ 85%

 Scenario: Arabica $4.40/lb | Hot Arabica ≥ 80%

 Scenario: Arabica $4.40/lb | Hot Arabica ≥ 70%

 Scenario: Arabica $4.40/lb | Hot Arabica ≥ 60%

 Scenario: Arabica $5.00/lb | Hot Arabica ≥ 85%

 Scenario: Arabica $5.00/lb | Hot Arabica ≥ 80%

 Scenario: Arabica $5.00/lb | Hot Arabica ≥ 70%

 Scenario: Arabica $5.00/lb | Hot Arabica ≥ 60%

 Scenario: Arabica $5.50/lb | Hot Arabica ≥ 85%

 Scenario: Arabica $5.50/lb | Hot Arabica ≥ 80%

 Scenario: Arabica $5.50/lb | Hot Arabica ≥ 70%

 Scenario: Arabica $5.50/lb | Hot Arabica ≥ 60%

 Scenario: Arabica $6.00/lb | Hot Arabica ≥ 85%

 Scenario: Arabica $6.00/lb | Hot Arabica ≥ 80%

 Scenario: Arabica $6.00/lb | Hot Arabica ≥ 70%

 Scenario: Arabica $6.00/lb | Hot Arabica ≥ 60%


,Arabica Price,Hot Arabica Min %,Total Cost,Arabica (lbs),Robusta (lbs),Arabica Cold %,Arabica Hot %,Storage (lbs),T_Land,T_Ocean,T_Air
0,4.4,0.85,16633100.0,467500.0,1732500.0,0.0,0.85,440000.0,220000.0,1760000.0,220000.0
1,4.4,0.80,16578100.0,440000.0,1760000.0,0.0,0.80,440000.0,220000.0,1760000.0,220000.0
2,4.4,0.70,16468100.0,385000.0,1815000.0,0.0,0.70,440000.0,220000.0,1760000.0,220000.0
3,4.4,0.60,16358100.0,330000.0,1870000.0,0.0,0.60,440000.0,220000.0,1760000.0,220000.0
4,5.0,0.85,16913600.0,467500.0,1732500.0,0.0,0.85,440000.0,220000.0,1760000.0,220000.0
5,5.0,0.80,16842100.0,440000.0,1760000.0,0.0,0.80,440000.0,220000.0,1760000.0,220000.0
6,5.0,0.70,16699100.0,385000.0,1815000.0,0.0,0.70,440000.0,220000.0,1760000.0,220000.0
7,5.0,0.60,16556100.0,330000.0,1870000.0,0.0,0.60,440000.0,220000.0,1760000.0,220000.0
8,5.5,0.85,17147350.0,467500.0,1732500.0,0.0,0.85,440000.0,220000.0,1760000.0,220000.0
9,5.5,0.80,17062100.0,440000.0,1760000.0,0.0,0.80,440000.0,220000.0,1760000.0,220000.0


The outcomes in this scenario analysis are heavily influenced by the thresholds we set in the model.

- Cold Arabica Cap: This constraint limits how much Arabica can be used in cold beverages. Since Arabica is more expensive, the model uses as little as allowed—often defaulting to 100% Robusta in cold drinks to minimize cost.

- Hot Arabica Minimum (e.g., 85%): This forces the model to use a high percentage of Arabica in hot drinks, regardless of price, which drives up total cost when Arabica prices rise.

- Fixed logistics constraints: Storage and transport volumes remain constant across all scenarios, so cost variation comes primarily from changes in bean sourcing.

**Key Insight**

These results don’t necessarily reflect the most realistic or balanced sourcing strategy—they reflect the cheapest possible solution under the current constraints.

To get more practical outcomes, Starbucks could adjust quality-related thresholds (like requiring more Arabica in cold drinks) and then re-run the scenarios to understand the cost vs. quality trade-off.



#Challenges

We translated Starbucks’ complex sourcing strategy into a simplified optimization model using public data. Since exact figures (like recipes, costs, and shipping rates) were not available, we estimated them from industry sources, which introduced uncertainty. We also had to carefully set thresholds—for example, requiring 85% Arabica in hot drinks and allowing more Robusta in cold drinks—to balance quality with cost.

These thresholds strongly shaped the model: because Arabica is more expensive, the solver minimized its use wherever possible. As a result, the model found the lowest-cost solution—but not necessarily the most realistic one.

The model also has limitations:

1. Additional rounds of refinement were required to ensure feasibility across multiple scenarios and to interpret shadow prices meaningfully. The current model also has structural limitations:

2. Single-period scope: It does not account for seasonality, inventory rollover, or dynamic planning.

3. Deterministic inputs: Demand and supply conditions are assumed fixed, without capturing uncertainty.

4. No disruption modeling: The model excludes risks like climate events, strikes, or geopolitical shocks.

5. Limited product and perishability scope: Only brewed coffee is considered, with no time-decay or shelf-life effects.

6. No labor or sustainability metrics: Workforce capacity and carbon emissions are not yet included.

Addressing these areas would enhance the model’s realism, robustness, and long-term strategic value.

---

#Next Steps

The model closely follows the thresholds we set. Since Arabica is more expensive, the solver minimized its use whenever possible—especially in cold drinks. This led to the lowest-cost solution, but not necessarily the most realistic or brand-aligned one. Adjusting these thresholds is key to exploring better quality-cost trade-offs.

To strengthen and scale the model, we propose the following enhancements:

1. **Validate Assumptions with Internal Data**
Refine key inputs—such as beverage composition, supplier pricing, and transport costs—using internal Starbucks data. This will improve model accuracy and decision relevance.

2. **Incorporate Regional Demand Forecasts**
Embedding geographical demand variation would allow optimization of sourcing and roasting strategies by region, enabling more efficient distribution and local responsiveness.

3. **Expand Model Scope to Sustainability Metrics**
Introduce carbon emissions, ethical sourcing targets, or ESG compliance as additional constraints or objectives. This supports Starbucks' corporate sustainability goals while enhancing decision alignment with long-term values.

4. **Leverage Advanced Analytics**
Integrate AI-based demand forecasting and supplier scorecards to automate and scale sourcing decisions. This could transform the model into a real-time decision-support tool for procurement and operations.

5. **Extend to Multi-Period and Stochastic Planning**
Moving from a static, annual model to a dynamic, multi-period framework would capture seasonality, inventory rollover, and contract lead times.
Adding stochastic elements (e.g., demand uncertainty, supply disruptions) would improve resilience and strategic foresight.



---

# **References**

1. Global Coffee Market Volatility: Lens on Supply Chain Disruption, January 2025

  https://www.chemanalyst.com/NewsAndDeals/NewsDetails/global-coffee-market-volatility-lens-on-supply-chain-disruption-35031

2. Direct Trade Sourcing Strategies for Specialty Coffee, May 2024, Kazaz, B. et al.

  https://pubsonline.informs.org/doi/abs/10.1287/msom.2021.0586?af=R

3. Designing an Efficient Supply Chain for Specialty Coffee from Caldas-Colombia, May 2020, Botero Lopez, S., & Chaudhry, M. S.

  https://dspace.mit.edu/handle/1721.1/126376

4. Making the Perfect Cold Brew Coffee, April 2022

  https://www.elgalabwater.com/blog/making-perfect-cold-brew-coffee

---

#GenAI Use

In this project, we used generative AI tools to support various aspects of our work. Below is a detailed account of how these tools were used:

* **Code Review and Debugging**: We used ChatGPT (GPT-4o) to improve our linear programming model, resolve coding issues, and add components for scenario analysis. It was especially helpful in writing clean code and making sure our model aligned with business logic.

* **Brainstorming Ideas**: ChatGPT assisted us in generating and refining ideas for our overall project structure, constraints, and what-if scenarios. It helped us think through trade-offs and make our approach more focused and effective.

* **Proofreading and Grammar Checks**: We used Grammarly to enhance the clarity, readability, and grammatical accuracy of our written content, including slides and final documentation.

Our team followed academic integrity guidelines throughout the process, carefully reviewing and validating all AI-generated content to ensure its accuracy and relevance.

---